In [49]:
#!pip install ensemble_boxes

In [50]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [51]:
# ensemble csv files
submission_files = [
    '/opt/ml/level2-object-detection-level2-cv-16/sample_submission/swin-t-l_1024.csv',
    '/opt/ml/level2-object-detection-level2-cv-16/sample_submission/swin-t-l_dyhead.csv',
    '/opt/ml/level2-object-detection-level2-cv-16/sample_submission/yolov5.csv',
    '/opt/ml/level2-object-detection-level2-cv-16/sample_submission/universenet.csv',
    '/opt/ml/level2-object-detection-level2-cv-16/sample_submission/yolox.csv'
] # submission lists
submission_df = [pd.read_csv(file) for file in submission_files]

In [52]:
image_ids = submission_df[0]['image_id'].tolist()

In [53]:
# ensemble 할 file의 image 정보를 불러오기 위한 json
annotation = '/opt/ml/dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [54]:
prediction_strings = []
file_names = []
# ensemble 시 설정할 iou threshold 이 부분을 바꿔가며 대회 metric에 알맞게 적용해봐요!
iou_thr = 0.4
skip_box_thr = 0.0001
weights = [0.6265, 0.4992, 0.5842, 0.4350, 0.4485]
sigma = 0.1

# 각 image id 별로 submission file에서 box좌표 추출
for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
#     각 submission file 별로 prediction box좌표 불러오기
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))

        # print(boxes_list[:5])
      
    # 예측 box가 있다면 이를 ensemble 수행
    if len(boxes_list):
        # boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        for box, score, label in zip(boxes, scores, labels):
            # prediction_string += str(label) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:76: UserWarning: Y1 < 0 in box. Set it to 0.
  warnings.warn('Y1 < 0 in box. Set it to 0.')
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:73: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:64: UserWarning: X1 < 0 in box. Set it to 0.
  warnings.warn('X1 < 0 in box. Set it to 0.')
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:85: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensembl

/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 0.0, 0.0, 0.8058671875].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 0.8644580078125, 0.0, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [1.0, 0.609462890625, 1.0, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 0.874310546875, 0.0, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88

/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 0.847423828125, 0.0, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 0.8829423828125, 0.0, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 0.829515625, 0.0, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 0.0, 0.0, 0.9341015625].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: U

In [55]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
# submission.to_csv('/opt/ml/level2-object-detection-level2-cv-16/sample_submission/final_ensemble_no_weight.csv')
submission.to_csv('/opt/ml/level2-object-detection-level2-cv-16/sample_submission/ensemble_wbf_with_weight.csv', index=False)
submission.head()

,PredictionString,image_id
0,7 0.9369266270359287 391.9505920410156 186.793...,test/0000.jpg
1,5 0.7395732901077406 132.7161102294922 0.19413...,test/0001.jpg
2,1 0.583296112375472 298.0648498535156 310.0496...,test/0002.jpg
3,9 0.5233601072368327 104.96698760986328 179.51...,test/0003.jpg
4,1 0.5692830796946702 199.1088409423828 252.381...,test/0004.jpg
